In [1]:
%cd ..

/home/prayson/Code/hadithi/dev.io


In [2]:
ls

assets/              joyapp/             pixi.lock    pyproject.toml  Users.csv
Books.csv            load-extensions.sh  pixi.toml    Ratings.csv
docker-compose.yaml  notebooks/          poetry.lock  README.md
installation.md      pg-Dockerfile       proj.env     solid.md


In [3]:
from pathlib import Path
from zipfile import ZipFile

from httpx import AsyncClient, Client
from mimesis import Person
from mimesis.locales import Locale
import jmespath
import pandas as pd

In [4]:
MIMESIS_LOCALE_URL = "https://mimesis.name/en/master/locales.html#overriding-locale"
COUNTRY_CODE_CSV = "assets/languages.csv"
[language_codes, *_] = pd.read_html(MIMESIS_LOCALE_URL)
country_codes = pd.read_csv(COUNTRY_CODE_CSV, usecols=["Country", "Two Letter"])

In [5]:
country_iso = (
    country_codes
     .transform(lambda d: d.str.lower())
     .rename(columns={"Country": "country", "Two Letter":"code"})
     #.drop_duplicates(subset=["country"], keep="first")
     .assign(code = lambda d: d["code"].str.split("-").str[0])
     .query("code != 'en'")
)

In [6]:
country_language = (language_codes[["Code", "Name"]]
     .transform(lambda d: d.str.lower())
     .merge(country_iso, how="left", left_on="Code", right_on="code")
     .dropna(subset=["country", "code"])
     .loc[:, ["code", "country"]]
     .set_index("country")
     .to_dict()
     .get("code")

)

In [7]:
DATA_DIR = Path("assets/books.zip")

with ZipFile(DATA_DIR) as z:
    print(z.namelist())
    files = (file for file in z.namelist() if Path(file).suffix == ".csv")
    data = {Path(file).stem.lower(): pd.read_csv(z.extract(file),low_memory=False) for file in files}     

['Books.csv', 'Ratings.csv', 'Users.csv', 'recsys_taxonomy2.png']


In [8]:

def get_name(language:str) -> str:
    person = Person(language)
    return person.full_name()


In [9]:
users = pd.read_csv("assets/users.csv", dtype_backend="pyarrow", dtype={"age":"Int64"})

In [10]:
users

,user_id,fake_name,location,age
0,1,Catrice Riggs,"nyc, new york, usa",<NA>
1,2,Johnson Colon,"stockton, california, usa",18
2,3,Лолита Кузьмин,"moscow, yukon territory, russia",<NA>
3,4,Abraim Melo,"porto, v.n.gaia, portugal",17
4,5,Barton King,"farnborough, hants, united kingdom",<NA>
...,...,...,...,...
278853,278854,Renaldo Gomez,"portland, oregon, usa",<NA>
278854,278855,Elizbeth Bird,"tacoma, washington, united kingdom",50
278855,278856,Arda Guichard,"brampton, ontario, canada",<NA>
278856,278857,Clifford Erickson,"knoxville, tennessee, usa",<NA>


In [13]:
data["books"]

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...
...,...,...,...,...,...,...,...,...
271355,0440400988,There's a Bat in Bunk Five,Paula Danziger,1988,Random House Childrens Pub (Mm),http://images.amazon.com/images/P/0440400988.0...,http://images.amazon.com/images/P/0440400988.0...,http://images.amazon.com/images/P/0440400988.0...
271356,0525447644,From One to One Hundred,Teri Sloat,1991,Dutton Books,http://images.amazon.com/images/P/0525447644.0...,http://images.amazon.com/images/P/0525447644.0...,http://images.amazon.com/images/P/0525447644.0...
271357,006008667X,Lily Dale : The True Story of the Town that Ta...,Christine Wicker,2004,HarperSanFrancisco,http://images.amazon.com/images/P/006008667X.0...,http://images.amazon.com/images/P/006008667X.0...,http://images.amazon.com/images/P/006008667X.0...
271358,0192126040,Republic (World's Classics),Plato,1996,Oxford University Press,http://images.amazon.com/images/P/0192126040.0...,http://images.amazon.com/images/P/0192126040.0...,http://images.amazon.com/images/P/0192126040.0...


In [30]:
client = Client(base_url="https://openlibrary.org/api")
ISBN = "0140301690" #"0091906814" #"0395974682" lord of the rings  #"0140301690" - Alice in Wonderland #ISBN=ASIN?
r = client.get(url="/books", params={"bibkeys": f"ISBN:{ISBN}", "format":"json"}, timeout=10)

In [31]:
b = r.json()
b

{'ISBN:0140301690': {'bib_key': 'ISBN:0140301690',
  'info_url': "https://openlibrary.org/books/OL26336110M/Alice's_Adventures_in_Wonderland",
  'preview': 'borrow',
  'preview_url': 'https://archive.org/details/isbn_0140301690',
  'thumbnail_url': 'https://covers.openlibrary.org/b/id/14544735-S.jpg'}}

In [32]:
open_client = Client(base_url="https://openlibrary.org")

In [33]:
[url, *_] = jmespath.search("*.info_url", b)
work = url.split("/")[-2]

r = open_client.get(f"/books/{work}.json", timeout=10)
r

<Response [200 OK]>

In [34]:
r.json()

{'notes': '"Text of this edition of Alice in Wonderland is that of the first edition."',
 'identifiers': {'google': ['9hOE1JPD3'], 'goodreads': ['11491198']},
 'title': "Alice's Adventures in Wonderland",
 'subtitle': 'and Through the Looking-Glass',
 'authors': [{'key': '/authors/OL22098A'}],
 'publish_date': '/1970',
 'other_titles': ['Through the looking-glass.'],
 'publishers': ['Puffin Books'],
 'covers': [14544735],
 'physical_format': 'Paperback',
 'full_title': "Alice's adventures in Wonderland and Through the looking glass",
 'publish_places': ['London, England'],
 'contributions': ['Tenniel, John, 1820-1914, illustrator'],
 'pagination': '345 pages :',
 'source_records': ['ia:isbn_0140301690', 'bwb:9780140301694'],
 'work_titles': ["Alice's adventures in Wonderland"],
 'languages': [{'key': '/languages/eng'}],
 'subjects': ['Fantasy'],
 'publish_country': 'enk',
 'by_statement': 'by Lewis Carroll ; with ninety-two illustrations by John Tenniel',
 'type': {'key': '/type/editio

In [36]:
b = r.json()

g = open_client.get(url=f'{b.get("works")[0].get("key")}.json', timeout=10)
g

<Response [200 OK]>

In [37]:
g.json()

{'title': "Alice's Adventures in Wonderland / Through the Looking Glass",
 'key': '/works/OL151411W',
 'authors': [{'author': {'key': '/authors/OL22098A'},
   'type': {'key': '/type/author_role'}}],
 'type': {'key': '/type/work'},
 'description': "A very real little girl named Alice follows a remarkable rabbit down a rabbit hole and steps through a looking-glass to come face to face with some of the strangest adventures and some of the oddest characters in all literature. The crusty Duchess, the Mad Hatter, the weeping Mock Turtle, the diabolical Queen of Hearts, the Cheshire-Cat, Tweedledum and Tweedledee--each one is more eccentric, and more entertaining, than the last. And all of them could only have come from the pen of Lewis Carroll, one of the few adults ever to enter successfully the children's world of make-believe--a wonderland where the impossible becomes possible, the unreal, real...where the heights of adventure are limited only by the depths of imagination.\r\n--back cover

In [158]:
# https://openlibrary.org/works/OL18020194W/bookshelves.json
# https://openlibrary.org/works/OL18020194W/ratings.json

In [22]:
open_client.get(url=f'{jmespath.search("works[0].key", b)}/bookshelves.json', timeout=10).json()

{'counts': {'want_to_read': 5596,
  'currently_reading': 520,
  'already_read': 219}}

In [23]:
open_client.get(url=f'{jmespath.search("works[0].key", b)}/ratings.json', timeout=10).json()

{'summary': {'average': 4.257731958762887,
  'count': 194,
  'sortable': 4.088522434697706},
 'counts': {'1': 12, '2': 6, '3': 21, '4': 36, '5': 119}}

In [24]:
jmespath.search("authors[0].author.key", g.json())

'/authors/OL107569A'

In [25]:
open_client.get(url=f'{jmespath.search("authors[0].author.key", g.json())}.json', timeout=10).json()

{'title': '(Carnegie, Dale, 1888-1955)',
 'key': '/authors/OL107569A',
 'name': 'Dale Carnegie',
 'type': {'key': '/type/author'},
 'death_date': 'November 1, 1955',
 'photos': [9160344, 7353715, 6732648],
 'personal_name': 'Dale Carnegie',
 'alternate_names': ['DALE CARNEGIE',
  'Dale carnegie',
  'Ka na ji (Carnegie, Dale, 1888-1955)',
  'Dale Breckenridge Carnegie',
  'Qia nai ji (Carnegie, Dale, 1888-1955)',
  'Dale CARNEGIE',
  'Ka nai ji (Carnegie, Dale',
  'dale carnegie',
  'Carnegie, Dale, 1888-1955.',
  'D. Carnegie',
  '(mei) Kanaiji (Carnegie, Dale)',
  'CARNEGIE DALE',
  'Dale Dale Carnegie',
  'CARNEGIE,DALE',
  'Carnegie, Dale, 1st',
  '(mei) Ka, Neiji (Carnegie, Dale',
  'Ka nei ji (Carnegie, Dale)',
  'Ka nei ji',
  'Carnegie, Dale',
  'dale-carnegie',
  'Carnegie Dale'],
 'birth_date': 'November 24, 1888',
 'source_records': ['ia:renxingderuodian0000meik',
  'amazon:2764023065',
  'amazon:9386037750',
  'amazon:6055092220',
  'bwb:9781519035554',
  'amazon:9350881241'

In [26]:
data["books"].sort_values("Year-Of-Publication")

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
212233,0947901051,Collect Post Office Cards,A. Roberts,0,Benham (A. Buckingham) Ltd,http://images.amazon.com/images/P/0947901051.0...,http://images.amazon.com/images/P/0947901051.0...,http://images.amazon.com/images/P/0947901051.0...
51621,0006479561,Sacred Clowns,Tony Hillerman,0,Fairmount Books Ltd Remainders,http://images.amazon.com/images/P/0006479561.0...,http://images.amazon.com/images/P/0006479561.0...,http://images.amazon.com/images/P/0006479561.0...
193438,0140382690,The Enemy You Killed,Peter McFarlane,0,Penguin Books Australia Ltd,http://images.amazon.com/images/P/0140382690.0...,http://images.amazon.com/images/P/0140382690.0...,http://images.amazon.com/images/P/0140382690.0...
25016,0879972734,Walkers On the Sky,David J Lake,0,Penguin Putnam~mass,http://images.amazon.com/images/P/0879972734.0...,http://images.amazon.com/images/P/0879972734.0...,http://images.amazon.com/images/P/0879972734.0...
17668,0590023888,Misty of Chincoteague,Marguerite Henry,0,Scholastic Inc,http://images.amazon.com/images/P/0590023888.0...,http://images.amazon.com/images/P/0590023888.0...,http://images.amazon.com/images/P/0590023888.0...
...,...,...,...,...,...,...,...,...
80264,0140301690,Alice's Adventures in Wonderland and Through t...,Lewis Carroll,2050,Puffin Books,http://images.amazon.com/images/P/0140301690.0...,http://images.amazon.com/images/P/0140301690.0...,http://images.amazon.com/images/P/0140301690.0...
97826,0140201092,Outline of European Architecture (Pelican S.),Nikolaus Pevsner,2050,Penguin USA,http://images.amazon.com/images/P/0140201092.0...,http://images.amazon.com/images/P/0140201092.0...,http://images.amazon.com/images/P/0140201092.0...
209538,078946697X,"DK Readers: Creating the X-Men, How It All Beg...",2000,DK Publishing Inc,http://images.amazon.com/images/P/078946697X.0...,http://images.amazon.com/images/P/078946697X.0...,http://images.amazon.com/images/P/078946697X.0...,NaN
221678,0789466953,"DK Readers: Creating the X-Men, How Comic Book...",2000,DK Publishing Inc,http://images.amazon.com/images/P/0789466953.0...,http://images.amazon.com/images/P/0789466953.0...,http://images.amazon.com/images/P/0789466953.0...,NaN


In [29]:
data["books"]

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...
...,...,...,...,...,...,...,...,...
271355,0440400988,There's a Bat in Bunk Five,Paula Danziger,1988,Random House Childrens Pub (Mm),http://images.amazon.com/images/P/0440400988.0...,http://images.amazon.com/images/P/0440400988.0...,http://images.amazon.com/images/P/0440400988.0...
271356,0525447644,From One to One Hundred,Teri Sloat,1991,Dutton Books,http://images.amazon.com/images/P/0525447644.0...,http://images.amazon.com/images/P/0525447644.0...,http://images.amazon.com/images/P/0525447644.0...
271357,006008667X,Lily Dale : The True Story of the Town that Ta...,Christine Wicker,2004,HarperSanFrancisco,http://images.amazon.com/images/P/006008667X.0...,http://images.amazon.com/images/P/006008667X.0...,http://images.amazon.com/images/P/006008667X.0...
271358,0192126040,Republic (World's Classics),Plato,1996,Oxford University Press,http://images.amazon.com/images/P/0192126040.0...,http://images.amazon.com/images/P/0192126040.0...,http://images.amazon.com/images/P/0192126040.0...
